# Reissner-Mindlin plate: boundary layers
In this notebook, we discuss the appearance of boundary layers when solving the Reissner-Mindlin plate equation.

In the limit $t\to 0$, the rotations $\beta$ are enforced to be $\nabla w$. When not handled correctly, shear locking occur. Even if a shear locking-free method is considered, allowing $\beta=\nabla w$ in the domain, the boundary conditions can be incompatible in the limit case. The reason is that we can prescribe two boundary conditions for the rotations at the boundary; however, only one for $\nabla w$, because the tangential derivative of $w$ is already fixed by $w$; only the normal derivative, $\partial_n w$, can be prescribed in the Kirchhoff-Love limit. Thus, a boundary layer may appear where the rotations rapidly change (from fulfilling the boundary conditions to being close to $\nabla w$ at the interior). Boundary layers are less significant for clamped and hard simply supported boundary conditions. Soft simply supported and free boundary conditions often lead to strong boundary layers. 

The boundary layer size is in the regime of the thickness, $\mathcal{O}(t)$. Reduced convergence may be observed if the mesh does not resolve this layer. One technique is to generate boundary layer meshes by placing small (anisotropic) elements. The optimal mesh depends on the thickness, but also the polynomial discretization order. Further, when having very anisotropic elements (high aspect ratio), rounding errors might reduce the overall accuracy.

Adaptive mesh refinement can be applied to identify the boundary layers. However, the number of elements is mostly too high, and the elements become very small quickly at the boundary layers. Thus, adaptive mesh refinement is commonly used with postprocessing techniques to change the mesh and account for the anisotropic behavior.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.meshes import MakeStructured2DMesh

In [ ]:
# length and thickness of plate
L  = 1
t  = 0.01

E  = 1e6
nu = 0.3
k  = 5/6
G = E/(2*(1+nu))

# force
q = sin(pi*x/L)

We consider an example taken from [<a href="https://onlinelibrary.wiley.com/doi/abs/10.1002/nme.5467">Sze, Hu. Assumed natural strain and stabilized quadrilateral Lobatto spectral elements for C0 plate/shell analysis. <i>Int. J. Numer. Meth. Engng.</i> 2017</a>], where an analytical solution is computed using MAPLE to solve the arising eigenvalue problem with high accuracy.



In [ ]:
def GetExSolution():
    w_ex = 0.112104526221152940265548393276936140028759596241*sin(pi*x) + 0.000632409771566098572843110743663530806201297944823*cosh(pi*y)*sin(pi*x) + 0.0137001636773051064646072440194929287608001683819*y*sinh(pi*y)*sin(pi*x) - 0.0000316122092964093846904903925214350233381599376977*sin(pi*x)*(0.0778005614549296617758704561411247100184621005913*cosh(pi*y) - 1.)
    bx = -0.352186756010538421893888815359115479436251630729*cos(pi*x) - 0.00198677389241045458752509183859938924329787256447*cosh(pi*y)*cos(pi*x) - 0.0430403335615994492112989782730640026686362137495*y*sinh(pi*y)*cos(pi*x) + 7.62933329113248462226480296578731364181334216691*10**(-72)*cosh(316.243370846569824755438139785423222466896713500*y)*cos(pi*x)
    by = -0.0156869375697155610521323358580923180040980409464*sinh(pi*y)*sin(pi*x) - 0.0430403335615994492112989782730640026686362137495*y*cosh(pi*y)*sin(pi*x) + 7.57905449687304576716760045185638770870950062624*10**(-74)*sinh(316.243370846569824755438139785423222466896713500*y)*sin(pi*x)
    return w_ex, CF( (bx,by) )

w_ex, b_ex = GetExSolution()
grad_w_ex = CF((w_ex.Diff(x), w_ex.Diff(y)))
grad_b_ex = CF((b_ex.Diff(x), b_ex.Diff(y)), dims=(2,2)).trans

def CMat(mat, E, nu):
    return E / (12 * (1 - nu**2)) * ((1 - nu) * mat + nu * Trace(mat) * Id(2))

sigma_ex = CMat(Sym(grad_b_ex), E, nu)
div_sigma_ex = CF((sigma_ex[0,0].Diff(x) + sigma_ex[0,1].Diff(y), sigma_ex[1,0].Diff(x) + sigma_ex[1,1].Diff(y)))

equ1 = -t**3*div_sigma_ex + t*k*G*(grad_w_ex+b_ex)
shear = k*G*t*(grad_w_ex+b_ex)
equ2 = -(shear[0].Diff(x) + shear[1].Diff(y)) - q 

Test that the exact solution is really a solution. The boundary layer can be seen best by the shear stress $\gamma=t\kappa G (\nabla w +\beta)$.

In [ ]:
def GenerateMesh(n=10, bl=False, layer_width=None, num_layer=None):
    if bl:
        scale = layer_width
        ratio = (num_layer/n)
                
        mapping = lambda x,y: (L/2*x,L/2*y/ratio*scale -L/2 if y < ratio else L/2*((1-scale)/(1-ratio)*y+(scale-ratio)/(1-ratio))-L/2)
    else:
        mapping = lambda x,y: (L/2*x,-L/2+L/2*y)
                
    return MakeStructured2DMesh(quads=False, nx=n, ny=n, mapping=mapping)

mesh = GenerateMesh(n=12, bl=True, layer_width=2*t, num_layer=4)
# draw shear stress to see the boundary layer
Draw(Norm(shear), mesh, "shear", order=4, deformation=True)
# test if the exact solution solves the equations
print(f"equ1 = {sqrt(Integrate(equ1*equ1, mesh,order=11))}")
print(f"equ2 = {sqrt(Integrate(equ2*equ2, mesh,order=11))}")

We use the shear locking free TDNNS method for Reissner-Mindlin plates <a href="https://link.springer.com/article/10.1007/s00211-017-0883-9">Pechstein and Schöberl. The TDNNS method for Reissner-Mindlin plates. <i> Numerische Mathematik 137</i>, 3 (2017), 713-740</a>].

In [ ]:
def MaterialInv(mat):
    mu  = E / 2 / (1+nu)
    lam = E * nu / (1-nu**2)
    return 1/(2*mu)*(mat-1/2*Trace(mat)*Id(2))+1/(4*(lam+mu))*Trace(mat)*Id(2)

def SolveRM_TDNNS(mesh, order=1):
    fesB = HCurl(mesh, order=order-1, dirichlet="left")
    fesS = Discontinuous(HDivDiv(mesh, order=order-1))
    fesH = NormalFacetFESpace(mesh, order=order-1, dirichlet="right|top")
    fesW = H1(mesh, order=order, dirichlet="left")

    fes = fesW*fesB*fesS*fesH
    (w,beta,sigma,alpha), (v,delta,tau, dalpha) = fes.TnT()
    
    n = specialcf.normal(2)
    
    a = BilinearForm(fes, symmetric=True, condense=True)
    a += (-12/t**3*InnerProduct(MaterialInv(sigma),tau) \
          + InnerProduct(tau,grad(beta)) + InnerProduct(sigma,grad(delta)))*dx
    a += ( -(sigma*n*n)*(delta*n + dalpha*n) - (tau*n*n)*(beta*n + alpha*n)  )*dx(element_boundary=True)
    a += k*G*t*InnerProduct( grad(w)+beta, grad(v)+delta )*dx

    f = LinearForm(fes)
    f += q*v*dx

    gfsol = GridFunction(fes)
    gfw, gfbeta, gfsigma, gfalpha = gfsol.components
    
    a.Assemble()
    f.Assemble()
    if a.condense:
        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfsol.vec
        r.data += a.harmonic_extension_trans * r
        gfsol.vec.data += a.mat.Inverse(fes.FreeDofs(True), inverse="sparsecholesky") * r
        gfsol.vec.data += a.harmonic_extension * gfsol.vec
        gfsol.vec.data += a.inner_solve * r
    else:
        inv = a.mat.Inverse(fes.FreeDofs())
        gfsol.vec.data = inv * f.vec
        
    
    return gfw, gfbeta

We make a convergence study where in one case uniform mesh refinement without a boundary layer mesh is considered and one where we place one fourth of the elements in the layer $2t$.

In [ ]:
order = 3
hs = [0.5**i for i in range(2,7)]
err_w = []
err_w_bl = []
err_b = []
err_b_bl = []

with TaskManager():
    for h in hs:
        n = int(L/h)
        # without boundary layer mesh
        mesh = GenerateMesh(n=int(L/h), bl=False)
        gfw, gfbeta = SolveRM_TDNNS(mesh, order=order)
        err_w.append(sqrt(Integrate((gfw-w_ex)**2+(Grad(gfw)-grad_w_ex)**2, mesh, order=11))/sqrt(Integrate(w_ex**2+grad_w_ex**2, mesh, order=11)))
        err_b.append(sqrt(Integrate((gfbeta-b_ex)**2, mesh, order=11))/sqrt(Integrate(b_ex**2, mesh, order=11)))
        # with boundary layer mesh
        mesh_bl = GenerateMesh(n=int(L/h), bl=True, layer_width=3*t, num_layer=n/4)
        gfw_bl, gfbeta_bl = SolveRM_TDNNS(mesh_bl, order=order)
        err_w_bl.append(sqrt(Integrate((gfw_bl-w_ex)**2+(Grad(gfw_bl)-grad_w_ex)**2, mesh, order=11))/sqrt(Integrate(w_ex**2+grad_w_ex**2, mesh, order=11)))
        err_b_bl.append(sqrt(Integrate((gfbeta_bl-b_ex)**2, mesh, order=11))/sqrt(Integrate(b_ex**2, mesh, order=11)))

    Draw(sqrt( (gfw-w_ex)**2 + (Grad(gfw)-grad_w_ex)**2), mesh)
    Draw(sqrt( (gfw_bl-w_ex)**2 + (Grad(gfw_bl)-grad_w_ex)**2), mesh_bl)


In [ ]:
import matplotlib.pyplot as plt

plt.yscale('log')
plt.xscale('log')
plt.xlabel("h")
plt.ylabel("relative error")
plt.plot(hs,err_w,"-*", label="$\|w-w_{\mathrm{ex}}\|$" )
plt.plot(hs,err_w_bl,"-x", label="$\|w_{bl}-w_{\mathrm{ex}}\|$" )
plt.plot(hs,err_b,"-*", label="$\|b-b_{\mathrm{ex}}\|$" )
plt.plot(hs,err_b_bl,"-x", label="$\|b_{bl}-b_{\mathrm{ex}}\|$" )
    
plt.plot(hs,[0.05*h**3 for h in hs],"--",color="k", label="$\mathcal{O}(h^3)$")
plt.legend()
plt.show()

We observe, that the convergence is nearly perfectly when using the boundary layer mesh. Without it, the convergence stagnates until the boundary layer gets resolved. The difference is about two magnitudes for the same amount of elements.